## Data Cleaning & Feature Engineering

Let's start cleaning our data. We will start this task by first looking at our data and then decide what we need to do. During this phase we will also assess if we need split or drop existing features, or create entirely new features.

For cleaning the data, I have created a .py containing all the functions that will help me clean. There can be other approaches, but this is what I'm going to follow.

A typical data cleaning task starts with reviewing the data which we have. So, let's start exploring our data and see what we have to clean.

In [106]:
#importing libraries

import pandas as pd
import data_cleaning as dc

In [107]:
# reading our properties data into dataframe

df_properties = pd.read_csv('properties1_rent.csv')

df_properties['price'] = df_properties.price.apply(lambda x: x.replace(' AED', ''))
df_properties['price'] = df_properties.price.apply(lambda x: x.replace('  / yearly', ''))  
#df_properties['no_of_bathrooms'] = df_properties.no_of_bathrooms.apply(lambda x: x if x == '7+' else x)    

# lets see the dimensions of the data
print('We have', df_properties.shape[0], 'rows and', df_properties.shape[1], 'columns in our dataframe')


We have 1893 rows and 12 columns in our dataframe


In [108]:
df_properties.head(3)

,id,title,price,location,latitude,longitude,type,size,no_of_bedrooms,no_of_bathrooms,completion_status,amenities
0,8935194,Luxury | Fitted Kitchen | Prime Location,34000,"Dubai, Arjan",25.065806,55.233004,Apartment,468sqft 43sqm,studio,1.0,NaN,"Unfurnished, Balcony, Built in Wardrobes, Cent..."
1,8922509,Luxurious Unit w/ Sea and Downtown Views,700000,"Dubai, Jumeirah, Jumeirah Bay Island",25.215192,55.235827,Apartment,1620sqft 151sqm,1,2.0,NaN,"Furnished, Balcony, Built in Wardrobes, Centra..."
2,8919569,With maid room /Spacious lay out /Balcony terrace,50000,"Dubai, Arjan",25.061118,55.245642,Apartment,900sqft 84sqm,1,2.0,NaN,"Unfurnished, Balcony, Built in Wardrobes, Cent..."


#### 1. Removing unwanted columns

Looking at the above data, we can see that we have a title and type columns, these does not add much value to our analysis and modeling. So we can safely remove them from our dataframe.

All other columns are relevant to either model building or exploratory analysis.

In [109]:
# calling our function to remove unwanted columns
column_list = ['title', 'type']
df_properties = dc.remove_unwanted_columns(df_properties, column_list)

# lets preview our data again after deletion
df_properties.head()

#delete row which have Nan amenities 
#df_properties['Charge_Per_Line'] = df_properties['Charge_Per_Line'].replace('-', np.nan)
df_properties  = df_properties.dropna(axis=0, subset=['amenities'])

#### 2. Removing duplicates

Let's make sure we don't have any duplicate properties. To do that, let's look at our id field for duplication.

In [110]:
pd.set_option('display.max_rows', 25)

property_ids = df_properties['id']
df_properties[property_ids.isin(property_ids[property_ids.duplicated()])].sort_values('id')

,id,price,location,latitude,longitude,size,no_of_bedrooms,no_of_bathrooms,completion_status,amenities
365,8790078,345000,"Dubai, Meadows",25.073219,55.155037,6264sqft 582sqm,5,5.0,NaN,"Unfurnished, Balcony, Built in Wardrobes, Maid..."
363,8790078,345000,"Dubai, Meadows",25.073219,55.155037,6264sqft 582sqm,5,5.0,NaN,"Unfurnished, Balcony, Built in Wardrobes, Maid..."
1449,8804419,40000,"Dubai, Dubai Sports City",25.040177,55.222445,833sqft 77sqm,1,2.0,NaN,"Unfurnished, Balcony, Built in Wardrobes, Cent..."
1453,8804419,40000,"Dubai, Dubai Sports City",25.040177,55.222445,833sqft 77sqm,1,2.0,NaN,"Unfurnished, Balcony, Built in Wardrobes, Cent..."
862,8810114,1800000,"Dubai, Mohammed Bin Rashid City, District One",25.168896,55.273708,16215sqft 1506sqm,6,7.0,NaN,"Unfurnished, Balcony, Built in Wardrobes, Cent..."
...,...,...,...,...,...,...,...,...,...,...
1257,8914938,95000,"Dubai, Bur Dubai",25.252613,55.299857,1441sqft 134sqm,2,2.0,NaN,"Unfurnished, Balcony, Built in Wardrobes, Pets..."
102,8924142,115000,"Dubai, Damac Hills 2, Aster",24.989435,55.388858,1882sqft 175sqm,4,5.0,NaN,"Furnished, Balcony, Built in Wardrobes, Covere..."
56,8924142,115000,"Dubai, Damac Hills 2, Aster",24.989435,55.388858,1882sqft 175sqm,4,5.0,NaN,"Furnished, Balcony, Built in Wardrobes, Covere..."
895,8937129,190000,"Dubai, Dubai Marina, Emaar 6 Towers",25.084146,55.148554,1552sqft 144sqm,2,2.0,NaN,"Unfurnished, Covered Parking, Lobby in Buildin..."


We can see above that we have 112 duplicate properties. Which means we have to remove 56 rows out of our properties data

In [111]:
# calling our custom function to remove duplicate properties on the basis of id

df_properties = dc.remove_duplocate_properties(df_properties)
property_ids = df_properties['id']
df_properties[property_ids.isin(property_ids[property_ids.duplicated()])].sort_values('id')

,id,price,location,latitude,longitude,size,no_of_bedrooms,no_of_bathrooms,completion_status,amenities


In [112]:
# As we can see now, we don't have any duplicates. Let's see the dimensions of our data

print('We have', df_properties.shape[0], 'rows and', df_properties.shape[1], 'columns in our dataframe')

We have 1674 rows and 10 columns in our dataframe


Comparing to our initial number of properties 1992, we now have 1935 unique properties. Which means we removed 57 redundent rows from our dataframe

#### 4. Removing NULL and unwanted values

Now let's see where we need to remove NULL data or unwanted data from our dataframe. 

To do this, we will first try to find our what data we have in each columns and then make a judgement on how to proceed with this task

##### 4.1 Price

In [113]:
# Unique values for price column

df_properties['price'].value_counts()

55000            45
Ask for price    43
75000            42
45000            39
30000            36
                 ..
3750000           1
325000            1
142000            1
276000            1
42500             1
Name: price, Length: 330, dtype: int64

In [114]:
# As we can see above that some of the properties have their price defined as 'Ask for price'. This is certinly not going to help us. So let's drop all the properties where price is defined as 'Ask for price'

df_properties = dc.remove_askforprice_properties(df_properties)
df_properties['price'].value_counts()

55000     45
75000     42
45000     39
30000     36
70000     34
          ..
69900      1
94999      1
590000     1
171000     1
68900      1
Name: price, Length: 329, dtype: int64

As you can see above, we have removed all the properties having price as 'Ask for price'

##### 4.2 Number of Bathrooms

In [115]:
df_properties['no_of_bathrooms'].value_counts()

2.0    537
1.0    499
3.0    292
4.0    135
5.0     89
6.0     38
7.0     35
Name: no_of_bathrooms, dtype: int64

In [116]:
# Now, before handline 19 NULL values in no_of_bathrooms, we will need to perform certain steps.

# fill nan with 0 in no_of_bathrooms

replacement= {'no_of_bathrooms': 0} 
df_properties = dc.replace_nan(df_properties, replacement)

# As no_of_bathrooms column is an object, lets convert it to int to get mean and average values
convert_dict = {'no_of_bathrooms' : int}

df_properties = dc.convert_datatypes(df_properties, convert_dict)

In [117]:
# now that bathrooms column is converted to int and null values are replaced with. Let's find out what is the mean value for bathroom and then use that value to replace NULL

print("Mean:", df_properties['no_of_bathrooms'].mean())
print("Rounded Mean number of bathrooms:", round(df_properties['no_of_bathrooms'].mean()))

Mean: 2.39546290619252
Rounded Mean number of bathrooms: 2


In [118]:
# Now lets replace 0 value with our mean value of 2

df_properties['no_of_bathrooms'].replace(0, round(df_properties['no_of_bathrooms'].mean()), inplace = True)

df_properties['no_of_bathrooms'].value_counts()

2    543
1    499
3    292
4    135
5     89
6     38
7     35
Name: no_of_bathrooms, dtype: int64

##### 4.3 Number of Bedrooms

In [119]:
df_properties['no_of_bedrooms'].value_counts()

1         528
2         393
studio    326
3         182
4         107
5          74
6          13
7           5
Name: no_of_bedrooms, dtype: int64

As we can see above, bedroom column includes variety of data. So:
- '+ Maid' will be removed from bedroom column and converted into a new feature call MaidRoom 0 or 1, indicating whether the apartment has a maid room or not
- studio will be converted to 0, as their really isn't any separate bedrooms in studio apartment

In [120]:
 # assigning 0 bedroom to studio apartments
# df_properties['no_of_bedrooms'] = df_properties.no_of_bedrooms.apply(lambda x: x.replace('studio', '0'))

# # removing + Maid string from bedrooms column
# df_properties['no_of_bedrooms'] = df_properties.no_of_bedrooms.apply(lambda x: x.replace(' + Maid', ''))
df_properties = dc.fix_bedroom_column(df_properties)
df_properties['id'].value_counts()

8816639    1
8924089    1
8936809    1
8335456    1
8926574    1
          ..
8805050    1
8805052    1
8870592    1
8889025    1
8916992    1
Name: id, Length: 1631, dtype: int64

##### 4.4 Amenities

For handling amenities, we will do:

2. Clean up the retreived amenities list of special chars, spaces, etc
3. Split the amenities list into individual feature with default value of 0
4. Loop all properties and set each amenity feature to either 1 or 0 if that apartment has that perticular amenity. I hope this makes sense :)

In [121]:
df_properties['amenities'] = df_properties['amenities'].astype(str)
most_amenities = max(df_properties['amenities'], key = len) #return the max amenities in dataframe
most_amenities

"Unfurnished, Balcony, Barbecue Area, Built in Wardrobes, Central A/C, Children's Play Area, Children's Pool, Concierge, Kitchen Appliances, Lobby in Building, Maid Service, Maids Room, Pets Allowed, Private Garden, Private Gym, Private Jacuzzi, Private Pool, Security, Shared Gym, Shared Pool, Shared Spa, Study, Vastu-compliant, View of Landmark, View of Water, Walk-in Closet, "

In [122]:
def fill_amenities(data_frame, new_amenities_list):
    data_frame.amenities = data_frame.amenities.apply(lambda x: x.replace(', ', ',').replace("'", '').replace(' ', '_').replace('-', '_').replace('/', '').lower())
    
    for index, prop_row in data_frame.iterrows():
        tab = []
        for val in prop_row['amenities'].split(','):
            for val1 in str(val).split(','):
                tab.append(val1)
        
        tab = tab[ : -1]
        #print(tab)        
        for obj in tab:
            for column_name in new_amenities_list:
                if obj.lower().strip() in column_name.lower().strip() or obj.lower().strip() == column_name.lower().strip():
                    data_frame.loc[index, column_name] = True
                # else:
                #     data_frame.loc[index, column_name] = False
            
    return data_frame;

In [123]:
# Retreiving MAX amenity from dataframe
df_properties['amenities'] = df_properties['amenities'].astype(str)
most_amenities = max(df_properties['amenities'], key = len)
print('most_amenities is {}'.format(most_amenities))
# Custom function for converting amenities to column
df_properties, new_amenities_list = dc.convert_amenities_to_columns(df_properties, most_amenities)

# Setting 1 or 0 against each property for their amenity
df_properties = fill_amenities(df_properties, new_amenities_list)

#df_properties.info()

most_amenities is ['unfurnished', 'balcony', 'barbecue_area', 'built_in_wardrobes', 'central_ac', 'childrens_play_area', 'childrens_pool', 'concierge', 'kitchen_appliances', 'lobby_in_building', 'maid_service', 'maids_room', 'pets_allowed', 'private_garden', 'private_gym', 'private_jacuzzi', 'private_pool', 'security', 'shared_gym', 'shared_pool', 'shared_spa', 'study', 'vastu_compliant', 'view_of_landmark', 'view_of_water', 'walk_in_closet', '']


##### 4.5 Apartment Quality

It is safe to say that based on number of amenities offered in an apartment, we can set the quality for each apartment.

But do note that we might miscatgorize the apartment just because the amenities are not recorded by property portal, even though it's highly priced apartment.

But as we are here for learning, so lets do this.

In [124]:
print('Maxiumum number of amenities any apartment has is ', len(most_amenities.split(', ')))

Maxiumum number of amenities any apartment has is  27


Based on above number we can rate our properties into following categories:
    
(1 - 7): Low<br>
(8 - 14): Medium<br>
(15 - 21): High<br>
(22 - 28): Ultra

In [125]:
df_properties = dc.mark_property_quality(df_properties)
df_properties.quality.value_counts()

Medium    1051
Low        317
High       253
Ultra       10
Name: quality, dtype: int64

As you can see we have successfully engineered a categorical feature for our property dataset

##### 4.6 Size

Let's lookinto size feature of our dataframe and clean-up the data. We will also engineer a new feature called price_per_sqft. This will be usefull for our exploratory analysis to see which localities are more expensive than others in terms of property pricings

In [126]:
df_properties['size'].value_counts()

450sqft  42sqm      24
800sqft  74sqm      19
700sqft  65sqm      15
900sqft  84sqm      13
850sqft  79sqm      12
                    ..
5489sqft  510sqm     1
855sqft  79sqm       1
1263sqft  117sqm     1
538sqft  50sqm       1
462sqft  43sqm       1
Name: size, Length: 959, dtype: int64

As you can see, size column contains details of both SQFT and SQM. For our analysis, we will keep the SQFT details and discard the SQM data, As keeping both values dosn't add value to our analysis

In [127]:
# Cleaning up property size details using custom function
df_properties = dc.cleanup_property_size(df_properties)

# rename column
df_properties.rename(columns={'size':'size_in_sqft'}, inplace = True)

# lets create a new computed
df_properties = dc.create_feature_price_per_sqft(df_properties)

df_properties.head(3)

,id,location,latitude,longitude,price,size_in_sqft,price_per_sqft,no_of_bedrooms,no_of_bathrooms,quality,...,security,shared_gym,shared_pool,shared_spa,study,vastu_compliant,view_of_landmark,view_of_water,walk_in_closet,
0,8935194,"Dubai, Arjan",25.065806,55.233004,34000,468,72.65,0,1,Medium,...,True,True,True,False,False,False,True,False,False,False
1,8922509,"Dubai, Jumeirah, Jumeirah Bay Island",25.215192,55.235827,700000,1620,432.10,1,2,Medium,...,False,True,True,False,False,False,False,False,False,False
2,8919569,"Dubai, Arjan",25.061118,55.245642,50000,900,55.56,1,2,Medium,...,True,True,True,False,False,False,True,False,False,False


##### 4.7 Completion Status

Let's take a look at completion status field and see what we find there

In [128]:
# Lets see what unique values we have.

df_properties.completion_status.value_counts()

Series([], Name: completion_status, dtype: int64)

In [129]:
# Lets count the null values

df_properties.completion_status.isna().sum()

1631

As we can see, we have significant amount of properties without completion status. This can taint the findings.

One way of handling this is to replace the empty completion status with 'Ready', as most of the properties are marked as ready.

Other way, is to eliminate the properties which have no completion status.

If we assess the completion status field, we can say that it is not that significant of a field to have the properties removed for it. So best way forward is to drop this column.

In [130]:
# Dropping completion_status field

df_properties.drop('completion_status', axis = 1, inplace = True)

In real world, the decision of dropping a column should be based on stakeholder input. We are just practicing, so why not :)

##### 4.8 Location

In [131]:
df_properties.location.value_counts()

Dubai, Jumeirah Village Circle                               99
Dubai, Dubai Marina                                          75
Dubai, Business Bay                                          63
Dubai, Arjan                                                 53
Dubai, Downtown Dubai                                        51
                                                             ..
Dubai, Dubai South (Dubai World Central), MAG 5               1
Dubai, Dubai Investment Park, Ritaj (Residential Complex)     1
Dubai, Dubai Marina, Oceanic                                  1
Dubai, Al Qusais                                              1
Dubai, Al Barari, Al Barari Villas                            1
Name: location, Length: 293, dtype: int64

If we look above, we can see that in some cases we have only have location as "Dubai, [Neighborhood Name]" and in same case we have "Dubai, [Neighborhood Name], Building/Project Name"

Let's standardize this by:

- Remove "Dubai, " part from location as we all know all properties are from Dubai
- Remove the project part of Location as this analysis is more targeted to locality rather than project or building
- All locations containing word downtown in neighbor, except for the ones called downtown jebel ali

In [132]:
df_properties = dc.clean_location_details(df_properties)

In [133]:
df_properties.neighborhood.unique()

array(['Arjan', 'Jumeirah', 'Jumeirah Village Circle', 'Business Bay',
       'Palm Jumeirah', 'Damac Hills 2', 'Downtown Dubai',
       'Arabian Ranches 2', 'Arabian Ranches', 'Dubai Sports City',
       'Jumeirah Village Triangle', 'Umm Suqeim',
       'Dubai Production City (IMPZ)', 'Al Jaddaf', 'Jumeirah Park',
       'Jumeirah Beach Residence', 'Dubai Hills Estate', 'Al Barari',
       'Dubai Creek Harbour (The Lagoons)', 'Dubai Harbour', 'Meydan',
       'Al Furjan', 'Bluewaters', 'Town Square', 'The Springs',
       'Dubai Silicon Oasis', 'DAMAC Hills', 'Remraam', 'Meadows',
       'International City', 'Nadd Al Sheba', 'Jumeirah Golf Estates',
       'Mohammed Bin Rashid City', 'Living Legends', 'Reem',
       'Discovery Gardens', 'Dubai Marina', 'Al Warsan', 'Motor City',
       'Al Barsha', 'Dubai South (Dubai World Central)', 'Greens',
       'Dubai Land', 'Barsha Heights (Tecom)', 'The Views',
       'Jumeirah Lake Towers', 'The Hills', 'DIFC', 'The Villa',
       'Culture 

In [134]:
df_properties.neighborhood.value_counts()

Dubai Marina               129
Jumeirah Village Circle    121
Downtown Dubai             119
Business Bay                77
Dubai Sports City           60
                          ... 
Serena                       1
Al Sufouh                    1
Mudon                        1
Al Garhoud                   1
Maritime City                1
Name: neighborhood, Length: 97, dtype: int64

Now we have a very normalized neighborhood data which can give us helpful insight of price trends per Dubai neighborhoods

#### 5. Saving Data

Now that we have cleaned our data, it's time to save this updated dataset into CSV format.

We will utilize this updated CSV in our exploratory and machine learning analysis

In [135]:
df_properties.to_csv('cleaned_property_data.csv', index = False)